### Question 1 (code required):
Provide 2 or 3 different ways to print out the IDs of the 3 users with logins closest to 2025-02-01 00:00 UTC.

We are not looking for the most optimal code, but a variety of methods you could use calculate the smallest difference and why you would select one method over another.

In [2]:
import json
from datetime import datetime
from heapq import nsmallest

class Question1:
    def __init__(self, records, cutoff_date):
        """
        Initializes the filter for the last logins.

        Args:
            records (list): List of dictionaries containing the last login IDs and dates.
            cutoff_date (str): Cutoff date in ISO8601 format ("%Y-%m-%dT%H:%M:%S%z").
        """
        self.records = records
        self.cutoff_datetime = datetime.strptime(cutoff_date, "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
        self.filtered_records = self._filter_records()
    
    def _filter_records(self):
        """
        - Filters the records to include only those whose last login occurred before the deadline.

        Returns:
            list: List of dictionaries containing 'id' and 'last_login' of the filtered users.
        """
        return [
            {"id": r["id"], "last_login": datetime.strptime(r["last_login"], "%Y-%m-%dT%H:%M:%SZ")}
            for r in self.records if datetime.strptime(r["last_login"], "%Y-%m-%dT%H:%M:%SZ") < self.cutoff_datetime
        ]
    
    def get_sorted_logins(self, limit=3):
        """
        - Returns the IDs of users with the most recent logins before the deadline,
        sorted from newest to oldest.

        Args:
            limit (int): Maximum number of IDs to return (default: 3).

        Returns:
            list: List of user IDs sorted by last login.
        """
        sorted_records = sorted(
            self.filtered_records,
            key=lambda x: x["last_login"],
            reverse=True
        )
        return [record["id"] for record in sorted_records[:limit]]
    
    def get_heap_logins(self, limit=3):
        """
        - Returns the IDs of users whose last logins are closest to the deadline,
          using the heap structure for efficiency.

        Args:
            limit (int): Maximum number of IDs to return (default: 3).

        Returns:
            list: List of IDs of users whose logins are closest to the deadline.
        """
        top_3_heap = nsmallest(
            limit, self.filtered_records, 
            key=lambda r: abs((r["last_login"] - self.cutoff_datetime).total_seconds())
        )
        return [r["id"] for r in top_3_heap]
    
    def get_iterative_logins(self, limit=3):
        """
        - Returns the IDs of users whose last logins are closest to the deadline,
          using an iterative method to find the closest ones.

        Args:
            limit (int): Maximum number of IDs to return (default: 3).

        Returns:
            list: List of IDs of users whose logins are closest to the deadline.
        """
        top_3_min = []
        remaining = self.filtered_records[:]
        for _ in range(limit):
            closest = min(remaining, key=lambda r: abs((r["last_login"] - self.cutoff_datetime).total_seconds()))
            top_3_min.append(closest)
            remaining.remove(closest)
        return [r["id"] for r in top_3_min]

def load_json(filename):
    """
    Loads a JSON file and returns its contents.

    Args:
        filename (str): Name of the JSON file.

    Returns:
        list: List of records loaded from the JSON.
    """
    with open(filename, "r", encoding="utf-8") as file:
        return json.load(file)

if __name__ == "__main__":
    records = load_json("../records.json")
    cutoff_date = "2025-02-01T00:00:00+0000"  # Data limite UTC
    filter_obj = Question1(records, cutoff_date)
    
    print("Sorted approach IDs:", filter_obj.get_sorted_logins())
    print("Heap approach IDs:", filter_obj.get_heap_logins())
    print("Iterative approach IDs:", filter_obj.get_iterative_logins())

Sorted approach IDs: ['d09c0ef6-8ea4-4c03-80bc-b0c113dbe28b', '41db84de-3d1c-416e-bf73-0218bbcd36c5', '3a895115-0e65-4372-a7d0-3b5dcac8095b']
Heap approach IDs: ['d09c0ef6-8ea4-4c03-80bc-b0c113dbe28b', '41db84de-3d1c-416e-bf73-0218bbcd36c5', '3a895115-0e65-4372-a7d0-3b5dcac8095b']
Iterative approach IDs: ['d09c0ef6-8ea4-4c03-80bc-b0c113dbe28b', '41db84de-3d1c-416e-bf73-0218bbcd36c5', '3a895115-0e65-4372-a7d0-3b5dcac8095b']
